In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly_resampler import register_plotly_resampler, FigureResampler, FigureWidgetResampler
from plotly.subplots import make_subplots
from scipy.signal import find_peaks, savgol_filter, oaconvolve, detrend, windows
from scipy.fft import fft, fftfreq, ifft
import matplotlib.pyplot as plt

register_plotly_resampler(mode='auto')

In [2]:
force = pd.read_csv('C:/Users/stickslip/Documents/leki-files/mcgill-natural-faults/surfaces/JBZQ04/JBZQ04-exp1-2023-06-16-kn.csv', header=1, 
                    names=['time', 'x_force', 'y_force'], usecols=[0,1,2])
displacement = pd.read_csv('C:/Users/stickslip/Documents/leki-files/mcgill-natural-faults/surfaces/JBZQ04/JBZQ04-exp1-2023-06-16-mm.csv', header=1, 
                           names=['time', 'x_disp', 'y_disp', 'y_motor_disp', 'x_motor_disp'], usecols=[0,1,2,3,4])


force_2 = pd.read_csv('C:/Users/stickslip/Documents/leki-files/mcgill-natural-faults/surfaces/JBZQ04/JBZQ04-exp2-2023-07-21-kn.csv', header=1, 
                      names=['time', 'x_force', 'y_force'], usecols=[0,1,2])
displacement_2 = pd.read_csv('C:/Users/stickslip/Documents/leki-files/mcgill-natural-faults/surfaces/JBZQ04/JBZQ04-exp2-2023-07-21-mm.csv', header=1, 
                             names=['time', 'x_disp', 'y_disp', 'y_motor_disp', 'x_motor_disp'], usecols=[0,1,2,3,4])


# force_sm1 = pd.read_csv('C:/Users/lekim/OneDrive/Desktop/School/research-2023/mcgill-natural-faults/surfaces/JULIASM1/SM1_force.csv', header=1, 
# names=['time', 'x_motor_disp', 'x_disp', 'friction'], usecols=[0,1,2,3])
# displacement_sm1 = pd.read_csv('C:/Users/lekim/OneDrive/Desktop/School/research-2023/mcgill-natural-faults/surfaces/JULIASM1/SM1_displacement.csv', header=1, 
# names=['time', 'x_motor_disp', 'x_disp', 'y_motor_disp', 'y_disp'], usecols=[0,1,2,3,4])

In [ ]:
# convert distances in sm1 from microns to mm
force_sm1['x_motor_disp'] = force_sm1['x_motor_disp'] / 1000
force_sm1['x_disp'] = force_sm1['x_disp'] / 1000

displacement_sm1['x_motor_disp'] = displacement_sm1['x_motor_disp'] / 1000
displacement_sm1['x_disp'] = displacement_sm1['x_disp'] / 1000
displacement_sm1['y_disp'] = displacement_sm1['y_disp'] / 1000
displacement_sm1['y_motor_disp'] = displacement_sm1['y_motor_disp'] / 1000

In [7]:
def add_cols(force, displacement):

    # Add friction (F / x = k)
    force['friction'] = -force['x_force'] / force['y_force']

    # Cropping data so friction is between 0 and 1
    displacement = displacement[force['friction'] > 0]
    force = force[force['friction'] > 0]

    displacement = displacement[force['friction'] < 1]
    force = force[force['friction'] < 1]

    # Mean velocity
    displacement['x_speed'] = np.gradient(displacement['x_disp'], displacement['time'])
    displacement['x_speed'] = displacement['x_speed'].rolling(50).mean()

    # Rate of change of friction wrt displacement
    force['friction_slope'] = np.gradient(force['friction'], displacement['x_motor_disp'])
    force['friction_accel'] = np.gradient(force['friction_slope'], displacement['x_motor_disp'])

    # Adding displacement columns to the force dataframe
    force['x_disp'] = displacement['x_disp']
    force['x_motor_disp'] = displacement['x_motor_disp']

    return force, displacement

def find_stress_drops_old(force):

    force['friction_slope'].replace([np.inf, -np.inf], np.nan, inplace=True)

    height_dev = force['friction_slope']

    force['peaks'] = False

    false_list = []

    for chunk in np.array_split(force, len(force) // 200 + 1):
        
        # Pandas series representing chunk
        z = chunk['friction_slope']

        z = savgol_filter(z, 15, 3)

        # detrend from scipy errors because of NaNs

        # Chunk-length list of Falses
        temp_list = np.full(len(chunk), False)

        # Find peaks
        peaks, _ = find_peaks(-z, distance=20, prominence=4*np.nanstd(-z))

        # Add one to index move to real peak
        peaks = peaks + 1

        # Replace False temp_list with True where peaks are
        temp_list[peaks] = True

        false_list = false_list + list(temp_list)
        print(len(false_list), len(temp_list))

    # height_dev = savgol_filter(height_dev, 15, 3)

    # filter = np.array([-1, -1, 8, -1, -1])

    # height_dev = oaconvolve(height_dev, filter, mode='same')

    # print(np.nanstd(height_dev))
    
    # peaks, _ = find_peaks(-height_dev, distance=30, prominence=20) # threshold=30, prominence=120

    # peaks = peaks + 1

    # force['peaks'] = False

    # force['peaks'].iloc[peaks] = True

    force['peaks'] = false_list

    stress_drops = force.loc[force['peaks'] == True]

    return stress_drops, 3

def convolve_stress_drops(force, window_size, stddev_coef):

    force['peaks'] = False

    # Loading the friction inverted (stress drops = inverted peaks)
    friction = force['friction']
    friction_slope = force['friction_slope']

    # Normalizing the inverted friction
    friction_slope = (friction_slope - np.min(friction_slope))/(np.max(friction_slope) - np.min(friction_slope))
    friction = (friction - np.min(friction))/(np.max(friction) - np.min(friction))

    # friction -= np.average(friction)

    # Appyling window to taper
    # friction = windows.tukey(len(friction), 0.05) * friction # , 0.1 for tukey

    # step = np.hstack((-1*np.ones(len(friction)), 1*np.ones(len(friction))))

    # friction_step = -1*np.convolve(friction, step, mode='valid')

    # friction_step = (friction_step - np.min(friction_step))/(np.max(friction_step)-np.min(friction_step))

    # Detect peaks of a certain prominence, and minimum distance from each other
    peaks, _ = find_peaks(-1*friction_slope, distance=20, prominence=0.1) # threshold=30, prominence=120

    # Calculate the standard dev of the friction slope data
    stddev = np.std(friction_slope)
    mean = np.mean(friction_slope)

    # Convert friction slope to numpy array
    friction_slope = friction_slope.to_numpy()
    friction = friction.to_numpy()

    # filter peaks < 1 std from mean, take initial length of list for drop %
    pre_len = len(peaks)
    peaks = [peak for peak in peaks if abs(friction_slope[peak] - mean) > stddev_coef*stddev]

    # For each index in peaks, check to see if there is a lower friction value in the last 30 samples (false pick)
    # Then search indices surrounding the peak pick to see if there are lower values (true drop)
    for peak in peaks:
        if peak >= 15:
            if np.min(friction[peak-15:peak]) < friction[peak]:
                peaks.remove(peak)

        if peaks != [] and peak in peaks and peak >= 3 and peak <= window_size - 4:
            # If the pick is too far forward (+ X +/-)
            # This deals with the flat case too (moved the stress drop to the left as far as possible)
            while friction[peak] - friction[peak - 1] >= 0 and friction[peak] - friction[peak + 1] <= 0:
                peaks.remove(peak) # Removing peak in peaks list
                peak = peak - 1 # Replacing peak with peak - 1
                peaks.append(peak) # Replacing peak value in peaks list (with peak - 1)
            
            while friction[peak] - friction[peak + 1] >= 0:
                peaks.remove(peak) # Removing peak in peaks list
                peak = peak + 1 # Replacing peak with peak - 1
                peaks.append(peak) # Replacing peak value in peaks list (with peak - 1)

    # print dropped # of peaks
    post_len = len(peaks)
    drop_count = pre_len - post_len
    print(f'Dopped {drop_count} peaks')

    # plt.plot(np.arange(0, len(friction_slope)), friction_slope)
    # plt.plot(np.arange(0, len(friction)), friction)
    # plt.scatter(peaks, friction[peaks])
    # plt.axhline(stddev_coef*stddev + mean)
    # plt.axhline(-stddev_coef*stddev + mean)
    # plt.show()

    # force['peaks'].iloc[peaks] = True

    # stress_drops = force.loc[force['peaks'] == True]

    return peaks

def fft_plot(data):
    
    # Number of points
    N = data.shape[0]

    # Sampling rate (# points / range of values, in points/distance)
    sampling_rate = N / (data.max(axis=0)['x_disp'] - data.min(axis=0)['x_disp'])
    T = 1 / sampling_rate # Sampling period

    # FFT of the raw friction data
    friction_fft = fft(data['friction'].values)

    # Frequencies of the FFT
    friction_fft_freq = fftfreq(N, T)

    # Filtering high frequencies and very low frequencies from the FFT
    friction_fft[np.where(np.logical_and(abs(friction_fft_freq)>=400, abs(friction_fft_freq)<=5000))] = 0
    # friction_fft[np.where(np.logical_and(abs(friction_fft_freq)>=2.45, abs(friction_fft_freq)<=2.5))] = 0

    # Inverse FFT to return smoothed signal to the time domain
    inverse_friction = ifft(friction_fft)

    # Applying Hann window to the smoothed (inversed) signal to kill edge artifacts (tapering)
    # inverse_friction = windows.tukey(len(inverse_friction), 0.05) * inverse_friction

    # Setting 'friction' column in dataframe to the smoothed signal
    data['friction'] = inverse_friction

    # plt.plot(np.arange(0, N), inverse_friction * 100)
    # plt.plot(friction_fft_freq, abs(friction_fft))
    # plt.ylim(-0.3, 0.3)
    # plt.xlim(0, 20)
    # plt.grid()
    # plt.show()

    return data

def find_stress_drops(force, window_size, stddev_coef):

    false_list = []

    for chunk in np.array_split(force, len(force) // window_size + 1):
        
        # Chunk-length list of Falses
        temp_list = np.full(len(chunk), False)
        
        # Find peaks
        peaks = convolve_stress_drops(chunk, window_size, stddev_coef)

        # Replace False temp_list with True where peaks are
        temp_list[peaks] = True

        false_list = false_list + list(temp_list)
        print(len(false_list))

    force['peaks'] = false_list

    stress_drops = force.loc[force['peaks'] == True]

    return stress_drops

In [18]:
# JBZQ04 1
force, displacement = add_cols(force, displacement)

stress_drops = find_stress_drops(force, 600, 3.3)

c:\Users\stickslip\Documents\leki-files\mcgill-natural-faults\lab-pc-env\Lib\site-packages\numpy\lib\function_base.py:1242: RuntimeWarning:

divide by zero encountered in divide

c:\Users\stickslip\Documents\leki-files\mcgill-natural-faults\lab-pc-env\Lib\site-packages\numpy\lib\function_base.py:1242: RuntimeWarning:

invalid value encountered in divide

c:\Users\stickslip\Documents\leki-files\mcgill-natural-faults\lab-pc-env\Lib\site-packages\numpy\lib\function_base.py:1243: RuntimeWarning:

divide by zero encountered in divide

c:\Users\stickslip\Documents\leki-files\mcgill-natural-faults\lab-pc-env\Lib\site-packages\numpy\lib\function_base.py:1243: RuntimeWarning:

invalid value encountered in divide

c:\Users\stickslip\Documents\leki-files\mcgill-natural-faults\lab-pc-env\Lib\site-packages\numpy\lib\function_base.py:1244: RuntimeWarning:

divide by zero encountered in divide

c:\Users\stickslip\Documents\leki-files\mcgill-natural-faults\lab-pc-env\Lib\site-packages\numpy\lib\functi

Dopped 5 peaks
600
Dopped 4 peaks
1200
Dopped 0 peaks
1800
Dopped 0 peaks
2400
Dopped 0 peaks
3000
Dopped 0 peaks
3600
Dopped 0 peaks
4200
Dopped 0 peaks
4800
Dopped 0 peaks
5400
Dopped 0 peaks
6000
Dopped 0 peaks
6600
Dopped 0 peaks
7200
Dopped 0 peaks
7800
Dopped 0 peaks
8400
Dopped 0 peaks
9000
Dopped 0 peaks
9600
Dopped 0 peaks
10200
Dopped 0 peaks
10800
Dopped 0 peaks
11400
Dopped 0 peaks
12000
Dopped 0 peaks
12600
Dopped 0 peaks
13200
Dopped 0 peaks
13800
Dopped 0 peaks
14400
Dopped 0 peaks
15000
Dopped 0 peaks
15600
Dopped 0 peaks
16200
Dopped 0 peaks
16800
Dopped 0 peaks
17400
Dopped 0 peaks
18000
Dopped 0 peaks
18600
Dopped 0 peaks
19200
Dopped 0 peaks
19800
Dopped 0 peaks
20400
Dopped 0 peaks
21000
Dopped 0 peaks
21600
Dopped 0 peaks
22200
Dopped 0 peaks
22800
Dopped 0 peaks
23400
Dopped 0 peaks
24000
Dopped 0 peaks
24600
Dopped 0 peaks
25200
Dopped 0 peaks
25800
Dopped 0 peaks
26400
Dopped 0 peaks
27000
Dopped 0 peaks
27600
Dopped 0 peaks
28200
Dopped 0 peaks
28800
Dopped 0 

C:\Users\stickslip\AppData\Local\Temp\ipykernel_6572\1485724507.py:92: RuntimeWarning:

invalid value encountered in scalar subtract



Dopped 0 peaks
200
Dopped 0 peaks
400
Dopped 0 peaks
600
Dopped 0 peaks
800
Dopped 0 peaks
1000
Dopped 0 peaks
1200
Dopped 0 peaks
1400
Dopped 0 peaks
1600
Dopped 0 peaks
1800
Dopped 0 peaks
2000
Dopped 0 peaks
2200
Dopped 0 peaks
2400
Dopped 0 peaks
2600
Dopped 0 peaks
2800
Dopped 0 peaks
3000
Dopped 0 peaks
3200
Dopped 0 peaks
3400
Dopped 0 peaks
3600
Dopped 0 peaks
3800
Dopped 0 peaks
4000
Dopped 0 peaks
4200
Dopped 0 peaks
4400
Dopped 0 peaks
4600
Dopped 0 peaks
4800
Dopped 0 peaks
5000
Dopped 0 peaks
5200
Dopped 0 peaks
5400
Dopped 0 peaks
5600
Dopped 0 peaks
5800
Dopped 0 peaks
6000
Dopped 0 peaks
6200
Dopped 0 peaks
6400
Dopped 0 peaks
6600
Dopped 0 peaks
6800
Dopped 0 peaks
7000
Dopped 0 peaks
7200
Dopped 0 peaks
7400
Dopped 0 peaks
7600
Dopped 0 peaks
7800
Dopped 0 peaks
8000
Dopped 0 peaks
8200
Dopped 0 peaks
8400
Dopped 0 peaks
8600
Dopped 0 peaks
8800
Dopped 0 peaks
9000
Dopped 0 peaks
9200
Dopped 0 peaks
9400
Dopped 0 peaks
9600
Dopped 0 peaks
9800
Dopped 0 peaks
10000
Dop

In [26]:
# JBZQ04 2
force_2, displacement_2 = add_cols(force_2, displacement_2)

stress_drops_2 = find_stress_drops(force_2, 1000, 2.2)

C:\Users\stickslip\AppData\Local\Temp\ipykernel_6572\1485724507.py:92: RuntimeWarning:

invalid value encountered in scalar subtract



Dopped 0 peaks
1000
Dopped 0 peaks
2000
Dopped 0 peaks
3000
Dopped 0 peaks
4000
Dopped 0 peaks
5000
Dopped 0 peaks
6000
Dopped 0 peaks
7000
Dopped 0 peaks
8000
Dopped 0 peaks
9000
Dopped 0 peaks
10000
Dopped 0 peaks
11000
Dopped 0 peaks
12000
Dopped 0 peaks
13000
Dopped 0 peaks
14000
Dopped 0 peaks
15000
Dopped 0 peaks
16000
Dopped 0 peaks
17000
Dopped 0 peaks
18000
Dopped 0 peaks
19000
Dopped 0 peaks
20000
Dopped 0 peaks
21000
Dopped 0 peaks
22000
Dopped 0 peaks
23000
Dopped 0 peaks
24000
Dopped 0 peaks
25000
Dopped 0 peaks
26000
Dopped 0 peaks
27000
Dopped 0 peaks
28000
Dopped 0 peaks
29000
Dopped 0 peaks
30000
Dopped 0 peaks
31000
Dopped 0 peaks
32000
Dopped 0 peaks
33000
Dopped 0 peaks
34000
Dopped 0 peaks
35000
Dopped 0 peaks
36000
Dopped 0 peaks
37000
Dopped 0 peaks
38000
Dopped 0 peaks
39000
Dopped 0 peaks
40000
Dopped 0 peaks
41000
Dopped 0 peaks
42000
Dopped 0 peaks
43000
Dopped 0 peaks
44000
Dopped 0 peaks
45000
Dopped 0 peaks
46000
Dopped 0 peaks
47000
Dopped 0 peaks
48000
D

In [4]:
lower = 13.924# 64.83# 29.16# 28.88#   25.6# # 
upper = 13.926# 64.93# 29.24# 28.98#   25.9# # 

subset_force = force[force['x_motor_disp'] < upper]
subset_force = subset_force[subset_force['x_motor_disp'] > lower]
subset_displacement = displacement[displacement['x_motor_disp'] < upper]
subset_displacement = subset_displacement[subset_displacement['x_motor_disp'] > lower]

stress_drops_new = convolve_stress_drops(subset_force, 100, 4)
stress_drops_fft = convolve_stress_drops(fft_plot(subset_force), 100, 4)
# fft_plot(subset_force)

KeyError: 'x_motor_disp'

In [28]:
# sort by column
sort_value = 'x_motor_disp'

force = force.sort_values(by=[sort_value])
displacement = displacement.sort_values(by=[sort_value])

force_2 = force_2.sort_values(by=[sort_value])
displacement_2 = displacement_2.sort_values(by=[sort_value])

stress_drops = stress_drops.sort_values(by=[sort_value])
stress_drops = stress_drops.drop_duplicates(subset=[sort_value])
stress_drops_2 = stress_drops_2.sort_values(by=[sort_value])
stress_drops_2 = stress_drops_2.drop_duplicates(subset=[sort_value])

# subset_force = subset_force.sort_values(by=[sort_value])
# subset_displacement = subset_displacement.sort_values(by=[sort_value])


In [29]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=force['x_motor_disp'], y=force['friction'], name='friction_1', connectgaps=True, mode='lines+markers'), secondary_y=False)

fig.add_trace(go.Scatter(x=stress_drops['x_motor_disp'], y=stress_drops['friction'], name='Stress drops', mode='markers'), secondary_y=False)

# fig.add_trace(go.Scatter(x=displacement['x_motor_disp'], y=force['friction_slope'], name='Friction rate', connectgaps=True, mode='lines+markers'), secondary_y=True)

# fig.add_trace(go.Scatter(x=displacement['x_motor_disp'], y=friction_filter, name='Friction rate Smoothed', connectgaps=True, mode='lines+markers'), secondary_y=True)


fig.add_trace(go.Scatter(x=force_2[sort_value], y=force_2['friction'], name='friction_2', connectgaps=True, mode='lines+markers'), secondary_y=False)

fig.add_trace(go.Scatter(x=stress_drops_2[sort_value], y=stress_drops_2['friction'], name='Stress drops 2', mode='markers'), secondary_y=False)

# fig.add_trace(go.Scatter(x=displacement_2['x_motor_disp'], y=force_2['friction_slope'], name='speed_2', connectgaps=True, mode='lines+markers'), secondary_y=True)

# fig.add_trace(go.Scatter(x=force_2['x_motor_disp'], y=force_2['friction_accel'], name='friction accel', connectgaps=True, mode='lines+markers'))

# fig.add_trace(go.Scatter(x=displacement_2['x_motor_disp'], y=friction_filter2, name='Friction rate Smoothed', connectgaps=True, mode='lines+markers'), secondary_y=True)


# Add steps as vertical lines
x_steps = np.array([2, 3, 4, 5, 8, 13])

t_steps = np.array([0, 100, 180, 230, 33, 360, 460, 520, 620, 740, 840, 1140, 1240, 1840, 2840, 3040, 3200, 3300, 3400, 3450, 3550, 3580, 3680, 3740, 3840, 3960, 4060, 4360, 4460, 5060, 5560])

# multiply by 50 to get time step
t_steps = t_steps + 162

fig.update_layout(width=1120, height=600, title='Friction vs. Displacement', xaxis_title='Displacement (mm)', yaxis_title='Friction Coefficient', font=dict(size=12))

fig


FigureWidgetResampler({
    'data': [{'connectgaps': True,
              'mode': 'lines+markers',
              'name': ('<b style="color:sandybrown">[R' ... 'tyle="color:#fc9944">~0.05</i>'),
              'type': 'scatter',
              'uid': '59f2560e-2c78-4e0c-ba5f-b5c24f3a1f32',
              'x': array([ 0.51237195,  0.55769377,  0.57224691, ..., 45.91700062, 45.98600045,
                          46.00000009]),
              'xaxis': 'x',
              'y': array([7.28812750e-05, 3.93361540e-03, 5.12082631e-03, ..., 6.73384328e-01,
                          6.76557671e-01, 6.65247369e-01]),
              'yaxis': 'y'},
             {'mode': 'markers',
              'name': 'Stress drops',
              'type': 'scatter',
              'uid': '1fd889d7-506d-4c5f-9dec-ed101f50baf0',
              'x': array([ 2.02015648,  2.07767519,  2.3105971 , ..., 45.81633955, 45.87583989,
                          45.94508186]),
              'xaxis': 'x',
              'y': array([0.284854